In [1]:
from autotst.reaction import *

In [2]:
rxn = AutoTST_Reaction("CC=C(C)C+[O]O_[CH2]C=C(C)C+OO", "H_Abstraction")

final_log_file = rxn.label + "_overall.log"

from ase.io.gaussian import *
optimized_ase_ts = read_gaussian_out(final_log_file)

rxn.ts.ase_ts = optimized_ase_ts
rxn.ts.update_from_ase_ts()


thermo.py:839 loadLibraries INFO Loading thermodynamics library from primaryThermoLibrary.py in /Users/nathan/Code/RMG-Py/../RMG-database/input/thermo/libraries...
thermo.py:839 loadLibraries INFO Loading thermodynamics library from thermo_DFT_CCSDTF12_BAC.py in /Users/nathan/Code/RMG-Py/../RMG-database/input/thermo/libraries...
thermo.py:839 loadLibraries INFO Loading thermodynamics library from CBS_QB3_1dHR.py in /Users/nathan/Code/RMG-Py/../RMG-database/input/thermo/libraries...
thermo.py:856 loadGroups INFO Loading thermodynamics group database from /Users/nathan/Code/RMG-Py/../RMG-database/input/thermo/groups...
transport.py:294 loadGroups INFO Loading transport group database from /Users/nathan/Code/RMG-Py/../RMG-database/input/transport/groups...
database.py:167 loadFamilies INFO Loading the user-specified kinetics families from /Users/nathan/Code/RMG-Py/../RMG-database/input/kinetics/families
statmech.py:526 loadGroups INFO Loading frequencies group database from /Users/nathan/

In [5]:
rxn.reactant_mols

[<AutoTST Molecule "CC=C(C)C">, <AutoTST Molecule "[O]O">]

In [6]:
from cclib.io import ccread
log_file_info = ccread(final_log_file)

rxn.vibrations = zip(log_file_info.vibfreqs, log_file_info.vibdisps)

got_imaginary_frequency = False
for vib, displacements in rxn.vibrations:
    if vib < 0: # Finding the imaginary frequency
        got_imaginary_frequency = True
        post_vibration_ts = rxn.ts.ase_ts.copy() 
        post_vibration_ts.arrays["positions"] -= displacements
        
assert got_imaginary_frequency == True, "Did not find an imaginary frequency"
def percent_change(original,new):
    percent_change = (abs(new - original) / original) * 100
    return percent_change

results = []

for torsion in rxn.ts.torsions:
    i,j,k,l = torsion.indices
    before = rxn.ts.ase_ts.get_dihedral(i,j,k,l)
    after = post_vibration_ts.get_dihedral(i,j,k,l)
    results.append(["Tor", torsion.indices, torsion.reaction_center, percent_change(before, after)])
    
for angle in rxn.ts.angles:
    i,j,k = angle.indices
    before = rxn.ts.ase_ts.get_angle(i,j,k)
    after = post_vibration_ts.get_angle(i,j,k)
    results.append(["Ang", angle.indices, angle.reaction_center, percent_change(before, after)])

for bond in rxn.ts.bonds:
    i,j = bond.indices
    before = rxn.ts.ase_ts.get_distance(i,j)
    after = post_vibration_ts.get_distance(i,j)
    results.append(["Bond", bond.indices, bond.reaction_center, percent_change(before, after)])

results = pd.DataFrame(results)
results.columns = ["type", "index", "center", "percent_change"]

if (np.log10(((results[results.center == "Yes"].mean()))) > np.log10(((results[results.center != "Yes"].mean()))) + 1).all():
    print "Vibrational analysis was successful"
    print True

else:
    print "Cannot reasonably say that we have arrived at a TS through vibrational analysis.\nRunning an IRC calc."
    print False

        

Vibrational analysis was successful
True


In [7]:
from autotst.calculators.vibrational_analysis import *

In [9]:
vib = Vibrational_Analysis(rxn)
print vib.validate_ts()
vib

vibrational_analysis.py:148 validate_ts INFO Vibrational analysis was successful


True


<AutoTST Vibrational Analysis "CC=C(C)C+[O]O_[CH2]C=C(C)C+OO">

In [10]:
from autotst.calculators.gaussian import *

In [11]:
calcs = AutoTST_Gaussian(rxn, ".")
calcs

<AutoTST Gaussian Calculators "CC=C(C)C+[O]O_[CH2]C=C(C)C+OO">

In [12]:
calcs.run_all()

gaussian.py:233 run_shell INFO Running shell optimization with center frozen...


RuntimeError: Gaussian in . returned an error: 127